# Apache Hive
É um sistema de data warehouse de código aberto, usado exclusivamente para consultar e analisar grandes conjuntos de dados armazenados no Hadoop.
O Hive utiliza uma linguagem chamada HiveQL (Hive Query Language) , que transforma as sentenças SQL em Jobs MapReduce executados no cluster Hadoop, para assim aproveitar as "skills" em SQL dos analistas e desenvolvedores do Facebook, que não eram na época tão proficientes em Java para usar o MapReduce.

## O que é?
- Interpreta instruções SQL para Job MapReducer
- Lê dados de arquivos estruturados e semi-estruturados no HDFS, e se baseia em metadados para simular uma tabela de um banco de dados relacional 
- Não possui um SGBD
- Foi desenhado para melhor performance analisando grandes quantidades de dados que se encontram em clusters.
- Data Warehouse do Hadoop

## Hive x RDBMS
|                              |     HIVE                    |     RDBMS                               |
|------------------------------|-----------------------------|-----------------------------------------|
|     Uso                      |     Foco   em analytics     |     Foco   em on line ou   analytics    |
|     Acesso                   |     Batch                   |     Batch   e Interativo                |
|     Integridade              |     Baixa                   |     Alta                                |
|     Escalabilidade           |     Horizontal              |     Vertical                            |
|     Armazenamento            |     Baixo   custo por PB    |     PB?                                 |
|     Interface                |     HiveQL                  |     SQL                                 |
|     Latência                 |     Minutos   ou mais       |     ms,   ml ou segundos                |
|     Estrutura   de dados     |     Não   Estruturado       |     Estruturado                         |

## Arquitetura
![Arquitetura](https://s3-sa-east-1.amazonaws.com/lcpi/c9506eb7-cbf9-4b2b-848d-284e36e363bc.png)



# Componentes do Hive
## Driver
Compila, otimiza, executa o HiveQL, e decide se executa a query local ou submeter em um job MapReduce.

## Metastore
Armazena os metadados, interpretando dos arquivos no HDFS como conteúdos de uma tabela. Armazena as informações de como as linhas e colunas são delimitadas dentro do arquivo (Hive Schemas). Pode ser armazenado no MySQL, Oracle, Derby ou Postgresql.

## CLI
Command Line Interface – para acessar o shell do hive. É o serviço padrão. Acessado pelo terminal no SO.

## Hiveserver
Permite que conexões (Thrift, ODBC e JDBC) de outros componentes tenham comunicação com o hive.

## Hiveserver2
Evolução do hiveserver, suportando autenticação e múltiplos usuários concorrentes.

## HiveQL
Utiliza Data Definition Language (DDL) para criar os bancos de dados e tabelas.

## Beeline
É a Command Line Interface para acessar o hiveserver2, utilizando conexão JDBC.

Execute comando beeline para iniciar a interface de consulta:
    
    beeline -u "jdbc:hive2://quickstart.cloudera:10000/default"
    
Após iniciar beeline, execute consulta no hive:

    show tables;

Para poder ver os comandos do beeline:

    beeline -help

# Tabelas

## Tabela Gerenciada (Manage Table)
Também conhecidas como **tabelas internas**, tem o ciclo de vida de seus dados controlados pelo Hive.

Armazena os dados dessas tabelas em um subdiretório sob o diretório definido por hive.metastore.warehouse.dir (por exemplo, / user / hive / warehouse) 

Lembre-se: quando excluímos a tabela os dados também são excluídos!

### Prática!
1. Criar diretório `/FileStore/tables/hive_aula` no hdfs.

    
    hdfs dfs -mkdir /FileStore
    hdfs dfs -mkdir /FileStore/tables
    hdfs dfs -mkdir /FileStore/tables/hive_aula/



2. Crie um arquivos chamado `tabela_interna.txt`, escreva algum conteúdo .

    1. vim tabela_interna.txt  
    2. insira algum texto interno  
    3. Esc -> :wq
    
ou use a `interface gráfica`.

3. Insira o arquivo no diretório do hive.

        hdfs dfs -put tabela_interna.txt /FileStore/tables/hive_aula/

ou use a `interface gráfica`.


4. Crie a tabela e faça o upload dos dados.

        CREATE TABLE ex_interna(texto_livre STRING);

        LOAD DATA INPATH '/FileStore/tables/hive_aula/tabela_interna.txt' INTO TABLE ex_interna

5. Consulte a tabela.

        select * from ex_interna;
    

## Tabela Externa(External Table)

External Table, é apenas uma referência a um arquivo, ou seja, o Hive não é dono dos dados. É possível, inclusive, que uma ou mais external tables apontem para o mesmo arquivo no HDFS.

Neste caso, quando excluímos a tabela os **dados não são excluídos**, apenas seus metadados! 

Por fim, é importante se atentar que é necessário informar a localização dos dados. 

A palavra-chave EXTERNAL diz ao Hive que esta tabela é externa  
Cláusula LOCATION é necessária para informar ao Hive onde os dados estão localizados

### Prática!
1. Criar diretorio no hdfs `/FileStore/tables/hive_aula_ext`.
  
        hdfs dfs -mkdir /FileStore/tables/hive_aula_externa
  
2. Crie o arquivo `tabela_externa.txt` no seu computador

        1. vim tabela_externa.txt  
        2. insira algum texto interno  
        3. Esc -> :wq

ou use a `interface gráfica`.
    
    
3. Crie a tabela com o comando do hiveQL.

       CREATE EXTERNAL TABLE ex_externa(texto_livre string) 
       LOCATION '/FileStore/tables/hive_aula_externa';  


4. Consulte a tabela

        select * from ex_externa

5. Copie o arquivo aula para novo diretório 

        hdfs dfs -rm /FileStore/tables/hive_aula_externa/tabela_externa.txt

6. Execute novamente a query a visualizar

        select * from ex_externa

## Location

Tabelas criadas sem LOCATION, os dados são armazenados no diretório default do HDFS hive que é: 
~~~
/user/hive/warehouse/NOME_DATABASE.db/NOME_TABELA 
~~~

Quando é criado no database default a estrutura é: 
~~~
/user/hive/warehouse/NOME_TABELA 
~~~

Tabelas criadas com LOCATION, o arquivo é armazenado neste local definido.

# Database
- Banco de dados no Hive é essencialmente apenas um catálogo ou espaço para tabelas .
- Se não especificar o banco de dados, o banco de dados `default` é usado. 
- Podemos criar vários bancos de dados .
- Podemos alterar o diretório na hora da criação.

## Comandos
Criar banco de dados
~~~ 
create database if not exists [nome] location [path no hdfs] ;
~~~

Visualizar banco de dados
~~~
show databases;
~~~

Definição do banco de dados 
~~~
describe database [nome];
~~~
Excluir banco de dados 
~~~
drop database [nome];
~~~

Alterar banco de dados
~~~
alter database [nome] [parâmetros];
~~~

### Prática!
1. Crie um banco de dados `aula` na pasta `/user/hive/db_aula` no Hive 

        create database aula location '/user/hive/db_aula'
    
2. Verifique o database

        show databases

3. Definição do banco de dados


        DESCRIBE DATABASE aula


## View

É uma referência lógica de uma consulta, é como salvar uma query em uma tabela lógica, para ser acessada de forma transparente como se fosse uma física. Muito utilizada para consultas preestabelecidas por usuários e controle de acesso de campos.




# Funções
O Hive disponibiliza funções nativas, UDF, UDAF e UDTF, além de permitir o desenvolvimento de funções customizadas.

UDF – Funções de usuário.
- Uma entrada e uma saída. Ex. SPLIT

UDAF – Funções de agregação.
- N entradas e uma saída. Ex. SUM

UDTF – Funções de tabelas.
- Uma entrada e N saídas. Ex. EXPLODE

## Função - SPLIT
Recebe um valor e um delimitador, devolve um array, ou seja, entra um elemento e sai N, no mesmo campo.

    
    
## Função - EXPLODE
Recebe um array, ou seja, um conjunto de valores delimitados, e devolve em linhas.





## Hue
Interface visual para manipulação do Hive e HDFS.

    http://quickstart.cloudera:8888

# SerDe
- Abreviação de Serializer / Deserializer. 
- O Hive usa a interface SerDe para IO (Input/Output)
- Um SerDe permite que o Hive leia os dados de uma tabela e os grave no HDFS em qualquer formato personalizado.
- Qualquer um pode escrever seu próprio SerDe para seus próprios formatos de dados. 
- Avro, ORC, RegEx, Parquet, CSV, JsonSerDe.

## Formatos de arquivos – Posicional 
Formato posicional utilizando SerDe. Crie um arquivo com o conteúdo abaixo no local do hdfs.

Arquivo:
~~~
00001cliente_01PF
00002cliente_02PJ
00003cliente_03PF
~~~

### Hive
~~~
create external table tabela_posicional ( conta string, nome string, tipo_pessoa string ) 
row format serde 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe' 
with SERDEPROPERTIES ('input.regex' ='(.{5})(.{10})(.{2})') 
location '/user/hive/tabela_posicional';
~~~

### Databricks

    DROP TABLE IF EXISTS tabela_posicional;
    create table tabela_posicional ( conta string, nome string, tipo_pessoa string ) 
    row format serde 'org.apache.hadoop.hive.serde2.RegexSerDe' 
    with SERDEPROPERTIES ('input.regex' ='(.{5})(.{10})(.{2})') 
    stored as textfile
    location '/FileStore/tables/tabela_posicional'

## Formatos de arquivos - Delimitado 
Formato posicional utilizando delimitador pipe. Crie um arquivo com o conteúdo abaixo no local do hdfs.

Arquivo:
~~~
1,cliente_01,PF
2,cliente_02,PJ
3,cliente_03,PF
~~~

### Hive
~~~
create external table tabela_delimitada( conta string, nome string, tipo_pessoa string ) 
row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile 
location '/user/hive/tabela_delimitada';
~~~

### Databricks

    create external table tabela_delimitada( conta string, nome string, tipo_pessoa string ) 
    row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile 
    location '/FileStore/tables/tabela_delimitada';


## Formatos de arquivos – JSON
JSON é um modelo para armazenamento de dados com capacidade de estruturar informações de uma forma bem mais compacta do que os arquivos sequenciais ou delimitados.

Crie a tabela: 
~~~
CREATE EXTERNAL TABLE tabela_json( campo string) stored as textfile LOCATION '/user/hive/tabela_json'; 
~~~

Crie um arquivo no HDFS /tabela_json com o conteúdo:
~~~
{"conta":1,"cliente":{"nome":"cliente_1","tipo_pessoa":"PF"}}
{"conta":2,"cliente":{"nome":"cliente_2","tipo_pessoa":"PJ"}}
{"conta":3,"cliente":{"nome":"cliente_3","tipo_pessoa":"PF"}}
~~~

Execute a query:
~~~
select get_json_object(campo, "$.conta") as conta, get_json_object(campo, "$.cliente.nome") as nome, get_json_object(campo, "$.cliente.tipo_pessoa") as tipo_pessoa from tabela_json;
~~~



# Particionamento no Hive
O Apache Hive organiza tabelas em partições. O particionamento é uma maneira de dividir uma tabela em partes relacionadas com base nos valores de colunas específicas, como data, cidade e departamento. Cada tabela na seção pode ter uma ou mais chaves de partição para identificar uma partição específica.

Características: 
- Particiona a tabela em diretórios no HDFS 
- Ganho de performance e eficiência 
- Evita a leitura da base toda 
- Muito utilizado em grandes volumes de dados

1. Vamos criar uma tabela particionada.

    create external table table_part(nome string) 
    partitioned by (ano int)
    row format delimited fields terminated by ',' 
    lines terminated by '\n'
    stored as textfile;


2. Verificar tabela.

    describe table_part;


3. Adicionando partição.

    alter table table_part 
    add partition(ano=2021) 
    location '/user/hive/table_part/particao/ano=2021' 


4. Excluindo partição.

    alter table table_part drop partition(ano=2021);


# Extração de dados Hive - HDFS

Agora vamos extrair dados do hive para hdfs.

    insert overwrite directory '/user/hive/output' select * from teste_load;


# Impala
O Hive não foi desenhado para executar queries em real time, com baixa latência, mas sim para melhor performance analisando grandes quantidades de dados que se encontram em clusters.

Cloudera lançou o Impala, permitindo execução de queries com baixa latência para dados armazenados no HDFS e no HBase que, usando a linguagem SQL, permite integração com ferramentas de BI como Tableau, Microstrategy, Pentaho, etc.

## Comandos Impala
Execute comando abaixo para abrir shell do impala
~~~
impala-shell
~~~

## Hue
O Impala também pode ser utilizado através do HUE, com interface visual, assim como o Hive!